# Worksheet 08

Name: Tristan Lee
UID: U24272030

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [12]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.387399385178682, 5.30477624129256, 4.814471200900233, 5.00288778878781, 5.515488691340493, 4.5224874355829074, 5.700253083794859, 4.870430782400645, 5.3384720159144745, 5.834922366071481]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [13]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[7.9402040261814735, 7.716514740204005, 6.545294703337508, 7.474234979305289, 8.908433364747292, 7.604131192330265, 9.495670671954688, 8.998645536789494, 7.132059973105156, 7.793000838488174]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [14]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p1 = s2.pop()
        data.append(p1)
print(data)

[7.793000838488174, 7.132059973105156, 8.998645536789494, 9.495670671954688, 5.834922366071481, 5.3384720159144745, 7.604131192330265, 8.908433364747292, 7.474234979305289, 6.545294703337508]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

We need the priors for both s1 and s2, and the mean and variances for the distributions of their weights (so 6 total parameters). We know the means and variances for each species since we initializes them, but not the priors.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [15]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.793000838488174, 7.132059973105156, 5.834922366071481, 5.3384720159144745, 7.604131192330265, 7.474234979305289, 6.545294703337508]
[8.998645536789494, 9.495670671954688, 8.908433364747292]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 6.817445152650335,  mean_2 = 9.134249857830492
var_1 = 0.7611025164627503,  var_2 = 0.06666887510519375


There are pretty close; P(S_i) are exactly right, the means are quite close, and the variances are pretty close too.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [16]:
from scipy.stats import norm
from math import sqrt

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.793000838488174
probability of observing that point if it came from cluster 0 =  0.24471470694782138
probability of observing that point if it came from cluster 1 =  2.135947849997728e-06
point =  7.132059973105156
probability of observing that point if it came from cluster 0 =  0.428497382655352
probability of observing that point if it came from cluster 1 =  1.3551872631496157e-13
point =  8.998645536789494
probability of observing that point if it came from cluster 0 =  0.020081948661373018
probability of observing that point if it came from cluster 1 =  1.3460314327499403
point =  9.495670671954688
probability of observing that point if it came from cluster 0 =  0.0041088301620048765
probability of observing that point if it came from cluster 1 =  0.5800824214066621
point =  5.834922366071481
probability of observing that point if it came from cluster 0 =  0.24253135141272814
probability of observing that point if it came from cluster 1 =  5.414732822949678e-36
point =  

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [17]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 6.849845102457459,  mean_2 = 9.138017637965763
var_1 = 0.8211841290447569,  var_2 = 0.06715192145063857


It looks like the estimates are further from what they should be. This is probably just because the random initialization of the species weights wasn't representative of the actual parameters.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)